<a href="https://colab.research.google.com/github/Patelhlt/text-summarizer/blob/main/extractive_summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
! pip install pandas scikit-learn transformers accelerate

In [12]:
!pip install kaggle
!pip install nltk
!pip install tensorflow
!pip install pandas

In [13]:
!pip install matplotlib-venn

In [14]:
!apt-get -qq install -y libfluidsynth1

E: Package 'libfluidsynth1' has no installation candidate


In [15]:
! pip install datasets rouge_score

importing libraries

Summarization using AutoTokenizer and AutoModelForSeq2SeqLM

1.For this Extractive summarization we are using Auto tokenizer and AutoModelForSeq2SeqLM classes and using t5-small as the model (Text-To-Text Transfer Trnsformer)

2.It accepts 60 million parameters while the base one accepts 660million parameters

3.More over it lightweight

In [16]:
import pandas as pd
import re
import string
from datasets import load_metric
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from transformers import BertTokenizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [17]:
path = '/content/drive/My Drive/cnndailymails.csv'
data=pd.read_csv(path)
data.head(5)


,id,article,highlights
0,92c514c913c0bdfe25341af9fd72b29db544099b,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...
1,2003841c7dc0e7c5b1a248f9cd536d727f27a45a,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...
2,91b7d2311527f5c2b63a65ca98d21d9c92485149,Dougie Freedman is on the verge of agreeing a ...,Nottingham Forest are close to extending Dougi...
3,caabf9cbdf96eb1410295a673e953d304391bfbb,Liverpool target Neto is also wanted by PSG an...,Fiorentina goalkeeper Neto has been linked wit...
4,3da746a7d9afcaa659088c8366ef6347fe6b53ea,Bruce Jenner will break his silence in a two-h...,"Tell-all interview with the reality TV star, 6..."


In [19]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments
import pandas as pd

Here we are reading dataset and converting it into Huggingface dataset format

In [20]:
data = data.drop(columns='id')

In [21]:
data.shape

(11490, 2)

In [22]:
data.head()

,article,highlights
0,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...
1,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...
2,Dougie Freedman is on the verge of agreeing a ...,Nottingham Forest are close to extending Dougi...
3,Liverpool target Neto is also wanted by PSG an...,Fiorentina goalkeeper Neto has been linked wit...
4,Bruce Jenner will break his silence in a two-h...,"Tell-all interview with the reality TV star, 6..."


In [23]:
print('ARTICLE:\n', data['article'][0])
print('=====================================================================================================================================================')
print('Summary:\n', data['highlights'][0])

ARTICLE:
 Ever noticed how plane seats appear to be getting smaller and smaller? With increasing numbers of people taking to the skies, some experts are questioning if having such packed out planes is putting passengers at risk. They say that the shrinking space on aeroplanes is not only uncomfortable - it's putting our health and safety in danger. More than squabbling over the arm rest, shrinking space on planes putting our health and safety in danger? This week, a U.S consumer advisory group set up by the Department of Transportation said at a public hearing that while the government is happy to set standards for animals flying on planes, it doesn't stipulate a minimum amount of space for humans. 'In a world where animals have more rights to space and food than humans,' said Charlie Leocha, consumer representative on the committee. 'It is time that the DOT and FAA take a stand for humane treatment of passengers.' But could crowding on planes lead to more serious issues than fighting 

\SPLITTING THE DATA INTO TRAIN DATA AND TEST DATA

In [24]:
from sklearn.model_selection import train_test_split

In [25]:
train_df, test_df = train_test_split(data, test_size=0.2, random_state=42)

In [26]:
print('Train Data ------->',len(train_df))
print('Test Data -------->',len(test_df))

Train Data -------> 9192
Test Data --------> 2298


In [27]:
print('Train Data ------->',train_df.columns)
print('Test Data -------->',test_df.columns)

Train Data -------> Index(['article', 'highlights'], dtype='object')
Test Data --------> Index(['article', 'highlights'], dtype='object')


In [28]:
datasets = [train_df, test_df]
dataset_names = ['Train Data', 'Test Data']
for i,dataset in enumerate(datasets):
    print(f'{dataset_names[i]} missing values:')
    print(dataset.isnull().sum())
    print('-------------------------')

Train Data missing values:
article       0
highlights    0
dtype: int64
-------------------------
Test Data missing values:
article       0
highlights    0
dtype: int64
-------------------------


PRE-PROCESSING

Converting all the text into lower case letters

In [29]:
train_df['article'] = train_df['article'].apply(lambda x: x.lower())
train_df['highlights'] = train_df['highlights'].apply(lambda x: x.lower())

In [30]:
train_df.head()

,article,highlights
8450,a dog breeder and her husband have been jailed...,warning: contains graphic content .\nmargaret ...
10407,"presidents, kings and dukes will gather in los...",microcon 2015 is the first north american gath...
7918,eric abidal believes that his former club barc...,barcelona through to champions league semi-fin...
8651,barcelona threw away a two-goal lead against s...,barcelona squandered a two-goal lead to stumbl...
10177,onlookers have described the horrifying moment...,a man is in a critical condition after a shark...


Removing Numbers, Special Characters and Punctuations

In [32]:
import re

def clean_text(text):
    # numbers
    text = re.sub(r'\d+', '', text)
    # Removing special characters and punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Removing extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

train_df['article'] = train_df['article'].apply(clean_text)
train_df['highlights'] = train_df['highlights'].apply(clean_text)

In [33]:
train_df.head()

,article,highlights
8450,a dog breeder and her husband have been jailed...,warning contains graphic content margaret and ...
10407,presidents kings and dukes will gather in los ...,microcon is the first north american gathering...
7918,eric abidal believes that his former club barc...,barcelona through to champions league semifina...
8651,barcelona threw away a twogoal lead against se...,barcelona squandered a twogoal lead to stumble...
10177,onlookers have described the horrifying moment...,a man is in a critical condition after a shark...


tokenization

In [34]:
import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt')

train_df['article'] = train_df['article'].apply(word_tokenize)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [35]:
train_df.head()

,article,highlights
8450,"[a, dog, breeder, and, her, husband, have, bee...",warning contains graphic content margaret and ...
10407,"[presidents, kings, and, dukes, will, gather, ...",microcon is the first north american gathering...
7918,"[eric, abidal, believes, that, his, former, cl...",barcelona through to champions league semifina...
8651,"[barcelona, threw, away, a, twogoal, lead, aga...",barcelona squandered a twogoal lead to stumble...
10177,"[onlookers, have, described, the, horrifying, ...",a man is in a critical condition after a shark...


In [38]:
train_df['highlights'][8651]

['b',
 'r',
 'c',
 'e',
 'l',
 'n',
 ' ',
 'q',
 'u',
 'n',
 'e',
 'r',
 'e',
 ' ',
 ' ',
 'w',
 'g',
 'l',
 ' ',
 'l',
 'e',
 ' ',
 ' ',
 'u',
 'b',
 'l',
 'e',
 ' ',
 ' ',
 ' ',
 'r',
 'w',
 ' ',
 ' ',
 'e',
 'v',
 'l',
 'l',
 ' ',
 'l',
 'n',
 'e',
 'l',
 ' ',
 'e',
 ' ',
 'n',
 ' ',
 'n',
 'e',
 'r',
 ' ',
 'b',
 'h',
 ' ',
 'c',
 'r',
 'e',
 ' ',
 'u',
 'n',
 'n',
 'n',
 'g',
 ' ',
 'g',
 'l',
 ' ',
 'n',
 ' ',
 'u',
 'r',
 ' ',
 'n',
 'g',
 'h',
 ' ',
 'e',
 'v',
 'l',
 'l',
 ' ',
 'h',
 ' ',
 'b',
 'c',
 'k',
 ' ',
 'w',
 'h',
 ' ',
 'g',
 'l',
 ' ',
 'f',
 'r',
 ' ',
 'e',
 'v',
 'e',
 'r',
 ' ',
 'b',
 'n',
 'e',
 'g',
 ' ',
 'n',
 ' ',
 'u',
 'b',
 ' ',
 'k',
 'e',
 'v',
 'n',
 ' ',
 'g',
 'e',
 'r',
 ' ',
 'l',
 ' ',
 'l',
 'g',
 ' ',
 'l',
 'e',
 'e',
 'r',
 ' ',
 'b',
 'r',
 'c',
 ' ',
 'n',
 'w',
 ' ',
 'w',
 ' ',
 'p',
 'n',
 ' ',
 'h',
 'e',
 ' ',
 'f',
 ' ',
 'r',
 'v',
 'l',
 ' ',
 'r',
 'e',
 'l',
 ' ',
 'r']

In [39]:
len(train_df['article'][8651])

701

In [40]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

train_df['article'] = train_df['article'].apply(lambda tokens: [word for word in tokens if word.lower() not in stop_words])
train_df['highlights'] = train_df['highlights'].apply(lambda tokens: [word for word in tokens if word.lower() not in stop_words])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [41]:
len(train_df['article'][8651])

701

In [42]:
len(train_df['highlights'][8651])

157

normal pre processing

In [43]:
def normal_processing(text:str):
    print('Normal processing\n\n'+text)
    #lower case
    lower_text=text.lower()

    #punctuation removing
    special_words_removed=re.sub(f'[{re.escape(string.punctuation)}]','',string=lower_text)

    #tokenizing
    tokenized_words=nltk.word_tokenize(special_words_removed)

    #remove stopwords
    stopwords_set=set(stopwords.words('english'))
    removed_stopwrods=[ word for word in tokenized_words if word not in stopwords_set ]

    #stemming and lemmatization
    stemmer=PorterStemmer()
    lemmatizer=WordNetLemmatizer()
    stemmed_words=[stemmer.stem(word) for word in removed_stopwrods]
    lemmaized_words=[lemmatizer.lemmatize(word) for word in stemmed_words]

    return lemmaized_words

bert preprocessing

In [44]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [45]:
from transformers import BertTokenizer

def berttokenize(text: str):
    return tokenizer(text, padding=True, truncation=True)

comparing

In [46]:
text = "Experts question if packed out planes are putting passengers at risk!"

print(normal_processing(text))
print(berttokenize(text))

Normal processing

Experts question if packed out planes are putting passengers at risk!
['expert', 'question', 'pack', 'plane', 'put', 'passeng', 'risk']
{'input_ids': [101, 8519, 3160, 2065, 8966, 2041, 9738, 2024, 5128, 5467, 2012, 3891, 999, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


we know for the models they won't accept strings as trainig inputs so we are converting the train dataset into numarables (Pre-processing)

In [47]:

# Preprocess the dataset
def preprocess_function(examples):
    inputs = [doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding='max_length')

    # Set up the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["highlights"], max_length=150, truncation=True, padding='max_length')

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [48]:
def preprocess_text(text):
 # Convert text to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenize the text into words
    words = text.split()
    # Remove stopwords
    stopwords = set(["the", "and", "is", "in", "to", "with", "a", "for", "of", "on", "it", "as", "at", "by"])
    words = [word for word in words if word not in stopwords]
    return words


In [49]:
# Function to summarize text
def summarize_text(text, max_length=150):
    input_ids = tokenizer.encode('summarize: ' + text, return_tensors='pt', max_length=512, truncation=True)
    summary_ids = model.generate(input_ids, max_length=max_length, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [50]:
file_link = 'https://drive.google.com/uc?export=download&id=1-_uAN_LoI2I9vaKbWhJOa9sKpt55CKbn'
df = pd.read_csv(file_link)

In [51]:
if 'preprocessed_article' not in df.columns:
    print("The column 'preprocessed_article' does not exist in the DataFrame.")
else:
    # If the column exists, proceed with summarization
    predicted_summary = [summarize_text(article) for article in df['preprocessed_article']]

The column 'preprocessed_article' does not exist in the DataFrame.


In [55]:
rouge = load_metric('rouge')

In [56]:
from datasets import load_metric

def generate_summary(data):
    summary=[]
    for dat in data['article']:

        summary.append(summarize(dat,min_length=30,max_length=90)[0]['summary_text'])



    data['predicted_summary']=summary

    return data

In [57]:
results=Dataset.from_dict(dataset[0:10])

In [59]:
results.to_pandas().head()

,article,highlights
0,Comedian Jenny Eclair travelled with her other...,The comedian stayed with Flavours who offer a ...
1,A woman of Arab and Jewish descent who was str...,The federal government will give Shoshana Hebs...
2,World No 1 Novak Djokovic has apologised to th...,Novak Djokovic beat Andy Murray 7-6 4-6 6-0 in...
3,(CNN)ISIS on Wednesday released more than 200 ...,Most of those released were women and children...
4,Hillary Clinton’s security detail arrived at a...,"Second modified, armored van spotted near Des ..."


In [60]:
# Loading ROUGE metric
rouge = load_metric("rouge")


references = results["highlights"]
predictions = results["article"]

rouge_output = rouge.compute(predictions=predictions, references=references)

for key in rouge_output:
    print(f"{key}: {rouge_output[key].high.fmeasure:.4f}")

rouge1: 0.2380
rouge2: 0.1479
rougeL: 0.1515
rougeLsum: 0.2082


In [61]:
rouge=load_metric('rouge')
rouge.add_batch(predictions=["article"],references=['highlights'])
scores=rouge.compute()

In [62]:
predictions = ["Your predicted summary here"]
references = ["Your actual reference summary here"]
rouge.add_batch(predictions=predictions, references=references)
# Compute the ROUGE scores
scores = rouge.compute()

# Print the ROUGE scores
print(scores)

{'rouge1': AggregateScore(low=Score(precision=0.75, recall=0.6, fmeasure=0.6666666666666665), mid=Score(precision=0.75, recall=0.6, fmeasure=0.6666666666666665), high=Score(precision=0.75, recall=0.6, fmeasure=0.6666666666666665)), 'rouge2': AggregateScore(low=Score(precision=0.3333333333333333, recall=0.25, fmeasure=0.28571428571428575), mid=Score(precision=0.3333333333333333, recall=0.25, fmeasure=0.28571428571428575), high=Score(precision=0.3333333333333333, recall=0.25, fmeasure=0.28571428571428575)), 'rougeL': AggregateScore(low=Score(precision=0.75, recall=0.6, fmeasure=0.6666666666666665), mid=Score(precision=0.75, recall=0.6, fmeasure=0.6666666666666665), high=Score(precision=0.75, recall=0.6, fmeasure=0.6666666666666665)), 'rougeLsum': AggregateScore(low=Score(precision=0.75, recall=0.6, fmeasure=0.6666666666666665), mid=Score(precision=0.75, recall=0.6, fmeasure=0.6666666666666665), high=Score(precision=0.75, recall=0.6, fmeasure=0.6666666666666665))}


In [63]:
def calculate_word_frequencies(words):
    word_frequencies = defaultdict(int)
    for word in words:
        word_frequencies[word] += 1
    return word_frequencies


In [64]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [65]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [66]:
def preprocess_text(text):
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Remove special characters
    text = text.lower()  # Convert to lowercase
    tokens = text.split()  # Tokenize
    tokens = [word for word in tokens if word not in stopwords.words('english')]  # Remove stop words
    return ' '.join(tokens)

In [67]:
# Load the pre-trained T5 model and tokenizer
model_name = 't5-small'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [68]:
# Preprocess the dataset
def preprocess_function(examples):
    inputs = [doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding='max_length')

    # Set up the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["highlights"], max_length=150, truncation=True, padding='max_length')

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [69]:
from transformers import pipeline
summarize=pipeline('summarization',model='t5-small',tokenizer=T5Tokenizer.from_pretrained(model_name))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [70]:
import os
print(os.getcwd())  # Prints the current working directory
print(os.listdir())  # Lists files in the current directory


/content
['.config', 'drive', 'sample_data']


In [71]:
dataset_path = 'path/to/your/cnndailymails.csv'


In [72]:
dataset_path = r'C:\path\to\your\cnndailymails.csv'


In [73]:
summarize(data['article'][0])[0]['summary_text']

"some experts are questioning if shrinking space on planes is putting our health and safety in danger . this week, a consumer advisory group set up by the department of transportation said that while the government is happy to set standards for animals flying on aircraft, it doesn't stipulate a minimum amount of space for humans . some airlines offer as little as 28 inches of space ."

In [74]:
# Function to summarize text
def summarize_text(text, max_length=150):
    input_ids = tokenizer.encode('summarize: ' + text, return_tensors='pt', max_length=512, truncation=True)
    summary_ids = model.generate(input_ids, max_length=max_length, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [77]:
df['generated_summary'] = 'article'

In [78]:
# Save the results to a new CSV file
output_file_path = 'generated_summaries.csv'
df.to_csv(output_file_path, index=False)